## Objetivo: Quanto um cliente de um determinado perfil está disposto a gastar?

A ideia é ter uma equação que permita alteração de parâmetros para avaliar o valor que um determinado cliente gastaria em um dado cenário (definido pelos parâmetros). Nesse sentido, nossa target é o campo "Total Pago (R$)".

O cenário é o conjunto de parâmetros que nós previamente escolhemos para avaliar o potencial de gasto. Em outras palavras, as variáveis que utilizaremos para prever a nossa target. Dessa forma, ele será definido pelos campos:
- Perfil do cliente
- Categoria de item em campanha de marketing
- Desconto total oferecido
- Tempo desde a última compra
- Tempo gasto para decisão

In [1]:
import numpy as np
import pandas as pd
from typing import List
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

BASE_PATH = "/media/bruno/Arquivos/Desenvolvimento/LaranjaLima"

## Preparação

Como nossa target é uma variável contínua, estamos diante de um problema de regressão. Antes de acessar os métodos de treino, precisamos garantir que as variáveis categóricas sejam devidamente traduzidas em números. Técnicas de encode como OneHot, Ordinal, Target e outras são úteis nesse ponto. Um cuidado que se deve ter é com relação ao modelo que será testado. Este notebook utiliza o OneHot por dois motivos: nosso modelo principal será o linear, que lida bem com esse tipo de encoder, e queremos deixar aberta a possibilidade de avaliar o impacto no modelo de árvore.

Em linhas gerais, este tratamento seria o suficiente para o processo de modelagem, uma vez que garantimos que todos os dados de entrada são numéricos. Todavia, ainda existe uma situação comum de ocorrer: campos com dados faltantes. O valor null indica que o valor não existe e pode ser gerado por alguma regra de negócio, falta de captura de informação ou um erro qualquer de processamento. De todo modo, os algoritmos de ajuste simplesmente executam cálculos, portando não vão conseguir lidar com null, afinal não é um número. Scalers são úteis nesse sentido, onde substituímos estes campos vazios por um valor padrão, que pode ser escolhido por meio de uma estratégia qualquer (número fixo, média, mediana, moda ou qualquer outro que julgar interessante). Apesar de estarmos "inserindo" informações que não são "originais" do conjunto de dados, estamos garantindo que os outros campos com números, que podem contribuir positivamente para o modelo, serão considerados no cálculo.

Por fim, ainda podemos realizar um outro tipo de processamento, que é alterar a escala dos valores de entrada. Esse tipo de transformação faz com que não se gerem coeficientes (em modelos que utilizam eles) com grande variação de escala entre si. Deste modo, alinhando com técnicas de regularização, garantimos que variações na entrada não gerem uma variação descontrolada na saída, uma vez que os coeficientes tendem a serem menores e sem muita desigualdade de escala entre eles.

## Processo

Com os dados, tudo que precisamos fazer é invocar os métodos de ajuste de modelo e teremos um pronto para utilizar. Porém, como saber se ele é bom? Para responder essa pergunta, precisaríamos testar este modelo com dados que já conhecemos, e não foram utilizados no ajuste, para saber se o modelo é efetivo, comparando o resultado dado pelo modelo e o valor real. Assim, a estratégia é pegar a nossa base original e repartir ela em duas. Uma parte nós vamos guardar e deixar quieta, ficará "esquecida" enquanto trabalhamos em todo o processo de treino na parte que vamos chamar de *treino*.

Em outra palavras, uma parte dos dados vamos guardar e, no momento oportuno, vamos tratar ela como se fossem dados novos que acabamos de obter. Todo o nosso processo de configuração de encode, definição de metodologia de substituição de null, padronização de valores e o ajuste do modelo serão feitos apenas com a parte de *treino*. As instâncias configuradas neste momento que serão utilizadas no fluxo de predição de novos valores, onde vamos resgatar aquela parte dos dados guardados para avaliar o quão bom o modelo foi.

Nosso processo então pode ser resumido nas seguintes etapas:
1. Carregar os dados existentes;
2. Separar os dados em duas partes, uma para treino e outra para teste;
3. Definir o campo que contêm a nossa target;
4. Definir os campos das variáveis preditoras (entrada do modelo), identificando o que é categórico e o que é numérico;
5. Ajustar as instâncias de encode, substituição e padronização utilizando apenas a base de treino;
6. Aplicar as transformações definidas anteriormente nos campos que representam as variáveis de entrada;
7. Instanciar o modelo que será testado, com as devidas particularidades (hyper parâmetros);
8. Utilizar os dados transformados em (6) para ajustar o modelo instanciado em (7);
9. Definir uma métrica de avaliação;
10. Recuperar os dados de teste e aplicar as mesmas transformações de (6);
11. Avaliar o resultado da métrica definida em (9) nos dados de teste e comparar com o resultado obtido no treino;
10. Caso o modelo seja bom, utilizar em novos dados, sempre aplicando as transformações de (6) antes de utilizar o modelo.

A classe criada abaixo tem como função implementar estes passos em alto nível, permitindo que o fluxo de execução se torne mais simples e o processo se torne mais fluido de executar, com as alterações mais importantes visíveis e o processo "maçante" encapsulado nos métodos.

In [19]:
class Pipe:
    def __init__(self, df: pd.DataFrame) -> None:
        self.__base_data = df
        self.__encoder = OneHotEncoder(sparse_output=False)
        self.__imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
        self.__scaler = StandardScaler()
        self.__categorical_columns: List[str] = None
        self.__numeric_columns: List[str] = None
        self.__target_column: str = None

    def set_categorical_columns(self, columns: List[str]) -> None:
        self.__categorical_columns = columns

    def set_numeric_columns(self, columns: List[str]) -> None:
        self.__numeric_columns = columns

    def set_target_column(self, column: str) -> None:
        self.__target_column = column

    def get_categorical_columns(self) -> List[str]:
        return [c for c in self.__categorical_columns]
    
    def get_numeric_columns(self) -> List[str]:
        return [c for c in self.__numeric_columns]

    def split(self, test=0.3) -> None:
        self.train, self.test = train_test_split(self.__base_data, test_size=test, random_state=666)

    def fit_preprocess(self) -> None:
        np_cat, np_num = self.train[self.__categorical_columns].values, self.train[self.__numeric_columns].values
        self.__encoder.fit(np_cat)
        self.__imputer.fit(np_num)
        self.__scaler.fit(np_num)

    def transform_data(self, df: pd.DataFrame) -> np.ndarray:
        np_cat, np_num = df[self.__categorical_columns].values, df[self.__numeric_columns].values
        np_cat = self.__encoder.transform(np_cat)
        np_num = self.__imputer.transform(np_num)
        np_num = self.__scaler.transform(np_num)
        return np.concatenate((np_cat, np_num), axis=1)
    
    def train_model(self, model) -> None:
        x_train = self.transform_data(self.train)
        y_train = self.train[self.__target_column].values
        model.fit(x_train, y_train)

        x_test = self.transform_data(self.test)
        y_test = self.test[self.__target_column].values

        print(f"MAE treino: {mean_absolute_error(y_train, model.predict(x_train))}")
        print(f"MAE teste: {mean_absolute_error(y_test, model.predict(x_test))}")

    def predict(self, df: pd.DataFrame, model) -> np.ndarray:
        x = self.transform_data(df)
        return model.predict(x)

## Trade off viés e variância

## Discutindo sobre os modelos testados

Comentar sobre as diferenças entre os modelos, focando no custo benefício do modelo linear e da "aproximação" feita pelas árvores no caso de regressão.

Comparar os valores da métrica no teste e no treino, falando sobre o esperado. Quando maior a diferença, maior o overfit e a variância. Comentar aqui sobre viês e variância, relacionando eles com a métrica adotada.

Comentar sobre o teste de overfit utilizando um SVM polinomial, relacionando com o polinômio interpolador.

Mostar o impacto do one hot no modelo de árvore, trocando o encode para target.

In [20]:
dataset = pd.read_csv(f"{BASE_PATH}/data/ml/ecommerce - python - gasto dos clientes.csv")
pipe = Pipe(dataset)
pipe.split(test=0.3)

pipe.set_target_column("Total Pago (R$)")
pipe.set_categorical_columns(["Perfil do Cliente", "Campanha em Curso"])
pipe.set_numeric_columns([
    "Desconto Total (%)",
    "Tempo desde a Última Compra (dias)",
    "Tempo para Finalizar Compra (min)"
])
pipe.fit_preprocess()

print("Testando modelo linear")
linear_model = SGDRegressor(loss="squared_error", penalty="l2", alpha=0.001, eta0=0.001, learning_rate="invscaling")
pipe.train_model(linear_model)

print("\nTestando modelo de máquinas de suporte")
svm_model = SVR(kernel="poly", C=1, epsilon=0.1, degree=5)
pipe.train_model(svm_model)

print("\nTestando modelo de árvore de decisão")
dt_model = DecisionTreeRegressor()
pipe.train_model(dt_model)

Testando modelo linear
MAE treino: 1219.7713183373567
MAE teste: 1260.3891670071464

Testando modelo de máquinas de suporte
MAE treino: 1246.5894557083
MAE teste: 1388.836666042846

Testando modelo de árvore de decisão
MAE treino: 0.0
MAE teste: 1632.0538924731184


## Utilizando o modelo

Ao passarmos por todo o processo de ajuste, podemos finalmente utilizar nosso modelo para ativamente prever o comportamento de um cliente qualquer em uma situação que definirmos. Em outras palavras, podemos testar cenários diversos para um mesmo cliente. Por exemplo, podemos variar o tempo gasto para fechar a compra ou o tempo desde a última vez que ele fez alguma compra. Isso pode gerar direcionamentos para o time de marketing, que vai saber passar a ter metas numéricas para perseguir e alcançar melhores resultados.

Por exemplo, tomando o nosso cliente mais "importante" (consome mais), será que ao reduzir o tempo de decisão na compra faz com que ele gaste mais? Sabendo que em média os consumidores deste perfil fecham suas compras com 15 minutos, conseguimos fazer eles comprarem mais se reduzirmos este tempo para 10? E se reduzirmos para 5? Assim, uma boa meta para o time de vendas poderia ser fazer com que esses clientes finalizem a compra mais rápido, esperando um aumento aproximado de R$ 100,00 por compra. Dessa forma, temos uma aplicação de ML dentro de um modelo de negócio, traçando com elas metas mais claras e objetivas. Isso é ser data driven, o que tando se fala recentemente.

In [31]:
columns = pipe.get_categorical_columns()
columns.extend(pipe.get_numeric_columns())
cases = [
    ["Médio ticket e alta frequência", "Eletrodomésticos", 1, 10, 15],
    ["Médio ticket e alta frequência", "Eletrodomésticos", 1, 10, 10],
    ["Médio ticket e alta frequência", "Eletrodomésticos", 1, 10, 5],

    ["Médio ticket e alta frequência", "Livros", 1, 10, 15],
    ["Médio ticket e alta frequência", "Livros", 1, 10, 10],
    ["Médio ticket e alta frequência", "Livros", 1, 10, 5],

    ["Médio ticket e alta frequência", "Livros", 1, 10, 15],
    ["Médio ticket e alta frequência", "Livros", 1, 20, 15],
    ["Médio ticket e alta frequência", "Livros", 1, 30, 15]
]
pipe.predict(pd.DataFrame(cases, columns=columns), linear_model)

array([1777.93234979, 1816.82286749, 1855.71338519, 1229.79858893,
       1268.68910663, 1307.57962433, 1229.79858893, 1231.75633529,
       1233.71408166])